# Running ASFMAUS

In [1]:
using StochasticDiffEq
using DiffEqCallbacks
using DifferentialEquations
using Plots
using LinearAlgebra
using DelimitedFiles
using DataFrames
using LinearAlgebra
using SparseArrays
using Distributions

In [2]:
include("/home/callum/ASF/Modules/ASF_input.jl");
include("/home/callum/ASF/Modules/ASF_model.jl");
include("/home/callum/ASF/Modules/ASF_analysis.jl");

In [3]:
input_path = "/home/callum/ASF/Inputs/"

"/home/callum/ASF/Inputs/"

In [4]:
input_1 = ASF_Inputs.Model_Data(input_path); #creating parameters to run model

-------------------------------------
3 Populations!
Enter Populations Seeded With ASF! 
(for multiple seperate with space)
-------------------------------------
stdin> 1
-------------------------------------
3 Populations!
Enter connections for each population
(for multiple seperate with space)
-------------------------------------
Population 1 Connects to:
stdin> 2
Population 2 Connects to:
stdin> 3
Population 3 Connects to:
stdin> 2
-------------------------------------
Strength of Population  1 to Population  2 Transmission:
stdin> 0.01
-------------------------------------
Strength of Population  2 to Population  3 Transmission:
stdin> 0.01


In [5]:
nt = input_1.Parameters.Populations.cum_sum[end] #total number of groups and farms
nc = 5 #number of classes
eqs = 11 #number of processes
n_ens = 100 #number of runs in ensemble

dc = sparse(zeros(nt*nc,nt*eqs))

#const _dc = zeros(nt*5,nt*10)

dc[0*nc*nt+1:nc*nt*eqs+nc:end] .= 1
dc[1*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[2*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+1:nc*nt*eqs+nc:end] .= 1

dc[2*nc*nt+2:nc*nt*eqs+nc:end] .= 1
dc[3*nc*nt+2:nc*nt*eqs+nc:end] .= -1
dc[4*nc*nt+2:nc*nt*eqs+nc:end] .= -1

dc[4*nc*nt+3:nc*nt*eqs+nc:end] .= 1
dc[5*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[6*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[7*nc*nt+3:nc*nt*eqs+nc:end] .= -1

dc[7*nc*nt+4:nc*nt*eqs+nc:end] .= 1
dc[8*nc*nt+4:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+4:nc*nt*eqs+nc:end] .= -1


dc[5*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[6*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[9*nc*nt+5:nc*nt*eqs+nc:end] .= -1;

In [6]:
function regular_c(du,u,p,t,counts,mark)  
    mul!(du,dc,counts)
end

function prob_func(prob,i,repeat) #function to change parameters while keeping network the same
                ASF_Model.reparam!(input_1)
                remake(prob,p=input_1.Parameters)
end 

prob_func (generic function with 1 method)

## Running the Model

### Frequency run

In [7]:
rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)

jumps = JumpSet(rj)

prob = DiscreteProblem(input_1.U0,input_1.Time,input_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)

#sol = solve(jump_prob, SimpleTauLeaping(),dt=1)

ensemble_prob_m = EnsembleProblem(jump_prob)#,prob_func=prob_func)
output = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= 10,dt=1);

In [8]:
ASF_Analysis.quick_analysis(output,input_1)

10×12 Matrix{Int64}:
 32  49  0  549  50   0   -2   -2  48  0  -2  -2
 36  28  0  554  50   0   -2   -2  49  0  -2  -2
 30  43  0  528  50   0   -2   -2  49  0  -2  -2
 50   1  0   49  49   0   -2   -2  48  0  -2  -2
 31  38  0  532  50   0   -2   -2  50  0  -2  -2
 27  42  0  553  50   0   -2   -2  48  0  -2  -2
 33  40  0  390  47   0   -2   -2  50  0  -2  -2
 36  34  0  474  48   0   -2   -2  50  0  -2  -2
 35  41  0  391  35  39  148  332  47  0  -2  -2
 30  43  0  340  33  41  178  639  50  0  -2  -2

## Running Multiple Networks

In [ ]:
function sensitivity(n_networks, n_dist, n_runs, path, name)
    
    input_folder = splitpath(path)[end]
    base_path = rsplit(path, input_folder)[1]

    #making save directory
    dir = "$(base_path)Results/$(name)"

    isdir(dir) || mkdir(dir)
    cd(dir)

    #copying inputs
    cp("$(path)", "Inputs",force = true)
    
    for i = 1:n_networks
        inputs = ASF_Inputs.Model_Data(path); #creating parameters to run model
        for j = 1:n_dist

            rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)
            jumps = JumpSet(rj)
            prob = DiscreteProblem(inputs.U0,inputs.Time,inputs.Parameters)
            jump_prob = JumpProblem(prob,Direct(),rj) 
            ensemble_prob_m = EnsembleProblem(jump_prob)
            outputs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1);

            summary = ASF_Analysis.quick_analysis(outputs, inputs) #update to new function from module (ASF_Model.quick_analysis(outputs))

            writedlm( "Results_$(i)_$(j).csv",  summary, ',')
            ASF_Model.reparam!(inputs) #re-doing params with
        end
        println("$(100*i/n_networks) %")

    end
    
end

In [ ]:
sensitivity(20,20,100,input_path, "Dataout") # running 

## Analysing Output

In [ ]:
density = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Baseline//");
connect_5 = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Connect_5//");


In [ ]:
density_r = ASF_Analysis.three_statistics(density,20, p_out = true);

In [ ]:
connect_5_r = ASF_Analysis.three_statistics(connect_5,20, p_out = true);